In [39]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_huggingface import ChatHuggingFace

In [40]:
embedding = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2')

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 920.71it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [41]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.",
             metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.",
             metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.",
             metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.",
             metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.",
             metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.",
             metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.",
             metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.",
             metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.",
             metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.",
             metadata={"source": "I5"}),
]

In [42]:
vectorstores = FAISS.from_documents(
    documents = all_docs,
    embedding=embedding

)

In [43]:
similarity_retriever = vectorstores.as_retriever(
    search_type = "similarity" , search_kwargs = {'k': 5}
)

In [48]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 50, "temperature": 0.7}
)

multisearch_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstores.as_retriever(search_kwargs={'k': 5}),
    llm=llm,
)


Loading weights: 100%|██████████| 148/148 [00:00<00:00, 640.12it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [49]:
query = "How to improve energy levels and maintain balance ?"
result1 = similarity_retriever.invoke(query)
result2 = multisearch_retriever.invoke(query)

Both `max_new_tokens` (=50) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [50]:
for i, doc in enumerate(result1):
    print(f"... Result {i+1} ...\n")
    print(f"Ans=> {doc.page_content}\n")

... Result 1 ...

Ans=> Drinking sufficient water throughout the day helps maintain metabolism and energy.

... Result 2 ...

Ans=> The solar energy system in modern homes helps balance electricity demand.

... Result 3 ...

Ans=> Consuming leafy greens and fruits helps detox the body and improve longevity.

... Result 4 ...

Ans=> Mindfulness and controlled breathing lower cortisol and improve mental clarity.

... Result 5 ...

Ans=> Photosynthesis enables plants to produce energy by converting sunlight.



In [51]:
for i, doc in enumerate(result2[:5]):  # Limit to first 5 results
    print(f"... Result {i+1} ...\n")
    print(f"Ans=> {doc.page_content}\n")

... Result 1 ...

Ans=> Python balances readability with power, making it a popular system design language.

... Result 2 ...

Ans=> Mindfulness and controlled breathing lower cortisol and improve mental clarity.

... Result 3 ...

Ans=> The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.

... Result 4 ...

Ans=> Drinking sufficient water throughout the day helps maintain metabolism and energy.

... Result 5 ...

Ans=> Photosynthesis enables plants to produce energy by converting sunlight.

